Underneath, in this notebook, you may find the prepared, e.t. filled with proper math and logic dataset, which performs greatly (>top 3%) even with untuned Deep Learning Model, which for this task is not suitable (obviously it do requires ML model, due to the highly dispersed low number of elements data datasets).

In [127]:
#Imports
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from google.colab.data_table import DataTable
pd.set_option('display.max_columns', None)
DataTable.max_columns = 30

In [128]:
# Getting the data from github
!wget -O test.csv https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/test.csv
!wget -O train.csv https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/train.csv
test_data = pd.read_csv('../content/test.csv')
train_data = pd.read_csv('../content/train.csv')

--2024-08-19 22:16:14--  https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28210 (28K) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  27.55K  --.-KB/s    in 0.002s  

2024-08-19 22:16:14 (11.7 MB/s) - ‘test.csv’ saved [28210/28210]

--2024-08-19 22:16:14--  https://raw.githubusercontent.com/LozinskiMatthew/Titanic-DL-Kaggle_Competition/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 

In [129]:
# Data cleansed
finding_patterns = pd.concat([train_data, test_data], axis=0)
train_data['Age'].fillna(value=finding_patterns['Age'].mean(), inplace=True)
train_data['Fare'].fillna(value=finding_patterns['Fare'].mean(), inplace=True)
test_data['Age'].fillna(value=finding_patterns['Age'].mean(), inplace=True)
test_data['Fare'].fillna(value=finding_patterns['Fare'].mean(), inplace=True)
train_data['Embarked'].fillna(value='X', inplace=True)
train_data['LetterInCabin'] = train_data['Cabin'].str[0]
test_data['LetterInCabin'] = test_data['Cabin'].str[0]
train_data.drop(['Cabin'], axis=1, inplace=True)
test_data.drop(['Cabin'], axis=1, inplace=True)
train_data['Name'].str.extract('([A-Za-z]+)\.', expand=True)
train_data['Name'] = train_data['Name'].str.extract('([A-Za-z]+)\.', expand=True)
train_data.loc[train_data['Age'] < 12, 'Name'] = 'Kid'
train_data['LetterInCabin'].fillna(value='N', inplace=True)
test_data['Name'] = test_data['Name'].str.extract('([A-Za-z]+)\.', expand=True)
test_data.loc[test_data['Age'] < 12, 'Name'] = 'Kid'
test_data['LetterInCabin'].fillna(value='N', inplace=True)

In [130]:
# Normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

ready_train_dataset_scaled = scaler.fit_transform(train_data.drop(['Sex', 'Survived', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1))
ready_train_dataset_scaled_df = pd.DataFrame(data=ready_train_dataset_scaled, columns=train_data.drop(['Sex', 'Survived', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1).columns)
train_data = pd.concat([train_data.drop(['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare'], axis=1), ready_train_dataset_scaled_df], axis=1)

ready_test_dataset_scaled = scaler.fit_transform(test_data.drop(['Sex', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1))
ready_test_dataset_scaled_df = pd.DataFrame(data=ready_test_dataset_scaled, columns=test_data.drop(['Sex', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1).columns)
test_data = pd.concat([test_data.drop(['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare'], axis=1), ready_test_dataset_scaled_df], axis=1)


In [131]:
# Standardization will be better if you were to decide to work with ML models, which will perform better

'''
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

ready_train_dataset_scaled = scaler.fit_transform(train_data.drop(['Sex', 'Survived', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1))
ready_train_dataset_scaled_df = pd.DataFrame(data=ready_train_dataset_scaled, columns=train_data.drop(['Sex', 'Survived', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1).columns)
train_data = pd.concat([train_data.drop(['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare'], axis=1), ready_train_dataset_scaled_df], axis=1)

ready_test_dataset_scaled = scaler.fit_transform(test_data.drop(['Sex', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1))
ready_test_dataset_scaled_df = pd.DataFrame(data=ready_test_dataset_scaled, columns=test_data.drop(['Sex', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1).columns)
test_data = pd.concat([test_data.drop(['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare'], axis=1), ready_test_dataset_scaled_df], axis=1)

'''

"\nfrom sklearn.preprocessing import StandardScaler\n\nscaler = StandardScaler()\n\nready_train_dataset_scaled = scaler.fit_transform(train_data.drop(['Sex', 'Survived', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1))\nready_train_dataset_scaled_df = pd.DataFrame(data=ready_train_dataset_scaled, columns=train_data.drop(['Sex', 'Survived', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1).columns)\ntrain_data = pd.concat([train_data.drop(['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare'], axis=1), ready_train_dataset_scaled_df], axis=1)\n\nready_test_dataset_scaled = scaler.fit_transform(test_data.drop(['Sex', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1))\nready_test_dataset_scaled_df = pd.DataFrame(data=ready_test_dataset_scaled, columns=test_data.drop(['Sex', 'Pclass', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1).columns)\ntest_data = pd.concat([test_data.drop(['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare'], axis=1), re

In [132]:
# OneHotEncoding
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)

embarked_encoded_train = ohe.fit_transform(train_data[['Embarked']])
embarked_train_df = pd.DataFrame(data=embarked_encoded_train, columns=ohe.categories_)
embarked_encoded_test = ohe.fit_transform(test_data[['Embarked']])
embarked_test_df = pd.DataFrame(data=embarked_encoded_test, columns=ohe.categories_)
pclass_encoded_train = ohe.fit_transform(train_data['Pclass'].values.reshape(len(train_data), 1))
pclass_train_df = pd.DataFrame(data=pclass_encoded_train, columns=['Low', 'Mid', 'High']) #For convinience and better comprehension
pclass_encoded_test = ohe.fit_transform(test_data['Pclass'].values.reshape(len(test_data), 1))
pclass_test_df = pd.DataFrame(data=pclass_encoded_test, columns=['Low', 'Mid', 'High'])
sex_encoded_train = ohe.fit_transform(train_data[['Sex']]) # Wow it is shortening for reshape in this context for y is 1!!
sex_train_df = pd.DataFrame(data=sex_encoded_train, columns=['Female', 'Male'])
sex_encoded_test = ohe.fit_transform(test_data[['Sex']])
sex_test_df = pd.DataFrame(data=sex_encoded_test, columns=['Female', 'Male'])
letter_encoded_train = ohe.fit_transform(train_data[['LetterInCabin']])
letter_train_df = pd.DataFrame(data=letter_encoded_train, columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'N'])
letter_encoded_test = ohe.fit_transform(test_data[['LetterInCabin']])
letter_test_df = pd.DataFrame(data=letter_encoded_test, columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'N'])
ticket_encoded_train = ohe.fit_transform(train_data[['Ticket']])
ticket_train_df = pd.DataFrame(data=ticket_encoded_train, columns=ohe.categories_)
ticket_encoded_test = ohe.fit_transform(test_data[['Ticket']])
ticket_test_df = pd.DataFrame(data=ticket_encoded_test, columns=ohe.categories_)
name_encoded_train = ohe.fit_transform(train_data[['Name']])
name_train_df = pd.DataFrame(data=name_encoded_train, columns=ohe.categories_)
name_encoded_test = ohe.fit_transform(test_data[['Name']])
name_test_df = pd.DataFrame(data=name_encoded_test, columns=ohe.categories_)
final_train_data = pd.concat([train_data.drop(['Pclass', 'Sex', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1), embarked_train_df, pclass_train_df, sex_train_df, letter_train_df, ticket_train_df, name_train_df], axis=1)
final_test_data = pd.concat([test_data.drop(['Pclass', 'Sex', 'Ticket', 'Embarked', 'LetterInCabin', 'Name'], axis=1), embarked_test_df, pclass_test_df, sex_test_df, letter_test_df, ticket_test_df, name_test_df], axis=1)
final_test_data = final_test_data.reindex(columns=final_train_data.columns, fill_value=0.0)
final_test_data.drop(columns=['Survived'], inplace=True)

In [133]:
# Float64 convergence, you may need it in some cases, and it does not hurt the algorithm, models do love numbers
final_train_data['SibSp'] =  final_train_data['SibSp'].astype('float64')
final_train_data['Parch'] =  final_train_data['Parch'].astype('float64')
final_train_data['Survived'] =  final_train_data['Survived'].astype('float64')

final_test_data['SibSp'] =  final_test_data['SibSp'].astype('float64')
final_test_data['Parch'] =  final_test_data['Parch'].astype('float64')
final_test_data['PassengerId'] =  final_test_data['PassengerId'].astype('float64')

In [134]:
# Creating final datasets to work with
final_train_data_labels = final_train_data.drop(columns=['Survived'])
final_train_data_target = final_train_data['Survived']
train_labels_np = final_train_data_labels.to_numpy()
train_target_np = final_train_data_target.to_numpy()
test_np = final_test_data.to_numpy()

In [135]:
#For training your models, experimenting with your models, you should use validation dataset, e.t. uncomment below, and use it for fit, and add proper callbacks
'''
divisor = np.round(float(len(final_train_data)) * 85 / 100)
divisor = int(divisor)
train_np_labels = final_train_data_labels[:divisor].to_numpy()
train_np_labels = final_train_data_target[:divisor].to_numpy()
valid_np_labels = final_train_data_labels[divisor:].to_numpy()
valid_np_target = final_train_data_target[divisor:].to_numpy()
'''

'\ndivisor = np.round(float(len(final_train_data)) * 85 / 100)\ndivisor = int(divisor)\ntrain_np_labels = final_train_data_labels[:divisor].to_numpy()\ntrain_np_labels = final_train_data_target[:divisor].to_numpy()\nvalid_np_labels = final_train_data_labels[divisor:].to_numpy()\nvalid_np_target = final_train_data_target[divisor:].to_numpy()\n'

In [136]:
# Just for you, func with Callbacks, were you to choose validation dataset, remember to uncomment these lines, tip. you can do it simultaneously via holding alt key, and pointing directions then ctrl+/
class CallbackCreator(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}:")
        print(f" - Loss: {logs['loss']}")
        print(f" - Accuracy: {logs['accuracy']}")
        print(f" - Precision: {logs['precision']}")
        # print(f" - Validation Loss: {logs['val_loss']}")
        # print(f" - Validation mae: {logs['val_accuracy']}")
        # print(f" = Validation Precision: {logs['val_precision']}")

That is the place for your model, were you to choose to go with DL, just write your model below, but in ML case, you should uncomment the standardization, and comment the normalization cell.

And beneath, lies a way to add to your model, final results, e.t. survived or not, just replace model_0 with  your current model name.

In [137]:
# Your model_0, random model that scores in top 3% with DL

model_0 = tf.keras.Sequential([
    tf.keras.layers.Dense(110, activation='relu'),
    tf.keras.layers.Dense(70, activation='relu'),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

loss = tf.keras.losses.BinaryFocalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model_0.compile(loss=loss, optimizer=optimizer, metrics=['accuracy', 'precision'])

model_0.fit(x=train_labels_np, y=train_target_np, epochs=120, batch_size=256, verbose=1, callbacks=CallbackCreator())


Epoch 1/120
1/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.3242 - loss: 0.1789 - precision: 0.3306Epoch 1:
 - Loss: 0.17569579184055328
 - Accuracy: 0.3748597204685211
 - Precision: 0.35947713255882263
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.3498 - loss: 0.1770 - precision: 0.3434
Epoch 2/120
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.6562 - loss: 0.1685 - precision: 0.5514Epoch 2:
 - Loss: 0.16581402719020844
 - Accuracy: 0.7418630719184875
 - Precision: 0.6842105388641357
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7132 - loss: 0.1669 - precision: 0.6389 
Epoch 3/120
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8086 - loss: 0.1594 - precision: 0.7857Epoch 3:
 - Loss: 0.155143603682518
 - Accuracy: 0.7991021275520325
 - Precision: 0.7879858613014221
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8035 - loss: 0.1565 - precision: 0.7876 
Epoch 4/120
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8086 - loss: 0.1506 - precision: 0.7708Epoch 4

In [138]:
test_data_v2 = pd.read_csv('../content/test.csv')
final_result = pd.DataFrame(data=test_data_v2['PassengerId'], columns=['PassengerId'])
test_length = len(final_test_data)
for i in range(test_length):
    input_data = test_np[i].reshape(1, -1)
    prediction = np.round(model_0.predict(input_data, verbose=1))
    final_result.loc[i, 'Survived'] = prediction
    print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[1.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.]]
1/1 ━━━━━━

At the end you can see how to save it to the file.

In [139]:
final_result.to_csv('my_submission.csv', index=False)

Have fun!!!